# Writing to local PostGIS db

In [ ]:
import pandas as pd
import os

In [ ]:
db_host = os.environ.get('DB_HOST')
db_port = os.environ.get('DB_PORT')
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')
db_name = os.environ.get('DB_NAME')

db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

In [ ]:
# read the csv file into a pandas dataframe, replacing NaN with None
df = pd.read_csv('final_output.csv', index_col=['cep_id'], dtype={'pa': 'Int64', 'eco': 'Int64', 'country': 'Int64'}).replace({pd.NA: None})
df.head()

,transition_0,transition_1,transition_2,transition_3,transition_4,transition_5,transition_6,transition_7,transition_8,transition_9,...,country,country_name,iso3,eco,eco_name,is_marine,pa,pa_name,is_protected,country_code
cep_id,,,,,,,,,,,,,,,,,,,,,
1,8.957921e+02,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,Lithuania,LTU,80412,Central European mixed forests,False,0,None,False,171.0
2,1.366843e+12,7.194092e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,0,None,False,NaN
4,0.000000e+00,1.836981e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,555547601,South Georgia and South Sandwich Islands Marin...,True,NaN
6,6.328776e+11,1.038784e+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,Area Beyond National Jurisdiction,ABNJ,2,Antarctic Polar Front,True,0,None,False,NaN
7,0.000000e+00,1.167926e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,Area Beyond National Jurisdiction,ABNJ,2,Antarctic Polar Front,True,345888,Terres Australes Françaises,True,NaN


In [ ]:
#create table from dataframe 
from sqlalchemy import create_engine
engine = create_engine(db_url)
df.to_sql('cep_water', engine, if_exists='replace')

526

In [ ]:
import psycopg2 as pg
from psycopg2 import sql
import pandas as pd


# connect to the database
conn = pg.connect(
    database=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)


In [ ]:
# create a cursor object using the cursor() method
cursor = conn.cursor()

# execute a statement
cursor.execute('SELECT * FROM cep_water LIMIT 5')

# print the result and close the connection
result = cursor.fetchall()
for row in result:
    print(row)

# close the cursor 
cursor.close()

(1, 895.792133, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 'Lithuania', 'LTU', 80412, 'Central European mixed forests', False, 0, None, False, 171.0)
(2, 1366843086810.8289, 7194092084292.959, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 'Area Beyond National Jurisdiction', 'ABNJ', 1, 'Antarctic', True, 0, None, False, None)
(4, 0.0, 1836981.041237, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 'Area Beyond National Jurisdiction', 'ABNJ', 1, 'Antarctic', True, 555547601, 'South Georgia and South Sandwich Islands Marine Protected Area', True, None)
(6, 632877592038.22, 10387835827531.256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 'Area Beyond National Jurisdiction', 'ABNJ', 2, 'Antarctic Polar Front', True, 0, None, False, None)
(7, 0.0, 11679257268.343046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 'Area Beyond National Jurisdiction', 'ABNJ', 2, 'Antarctic Polar Front', True, 345888, 'Terres Australes Françaises', True, None)


In [ ]:
def get_country_areas(conn, is_pa=False, is_marine=False):
    try:
        # create a cursor object using the cursor() method
        cursor = conn.cursor()
        # query all rows transition_0  to transition_10, filter where pa is 0 and where marine is false, and group by country_name and sum the areas for each transition band
        bands_SUM = [f'SUM(transition_{i}) as "transition_{i}"' for i in range(11)]
        query = sql.SQL('SELECT country_name, {bands} FROM cep_water WHERE is_protected = {is_pa} AND is_marine = {is_marine} GROUP BY country_name').format(
            bands=sql.SQL(', ').join(map(sql.SQL, bands_SUM)),
            is_pa=sql.Literal(is_pa),
            is_marine=sql.Literal(is_marine)
        )
        # #print query as string
        # print(query.as_string(conn))                                                                                                                    
        # filter where 
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except Exception as e:
        print("error: ", e)
    finally:
        conn.rollback()

In [ ]:
bands= [f'transition_{i}' for i in range(11)]
df_protected = pd.DataFrame(get_country_areas(conn, is_pa=True, is_marine=False), columns=['country']  + bands).set_index('country')
df_unprotected = pd.DataFrame(get_country_areas(conn, is_pa=False, is_marine=False), columns=['country']  + bands).set_index('country')

In [ ]:
df_protected

,transition_0,transition_1,transition_2,transition_3,transition_4,transition_5,transition_6,transition_7,transition_8,transition_9,transition_10
country,,,,,,,,,,,
Afghanistan,2.310425e+10,1.434245e+08,6.337323e+07,1.342069e+07,4.948676e+07,1.551678e+08,7.220795e+07,5.306655e+06,1.976603e+07,2.081170e+07,1.703576e+08
Albania,5.189864e+09,6.605394e+08,2.228370e+07,7.219132e+06,1.903888e+07,2.773176e+07,8.607468e+06,2.080579e+06,7.619051e+06,1.627974e+06,1.154378e+07
Algeria,1.334345e+12,5.696404e+07,1.256128e+07,1.529621e+06,5.371571e+08,1.493158e+09,2.222218e+08,4.094169e+06,5.686030e+06,8.596214e+06,1.090141e+09
American Samoa,2.029997e+07,1.260135e+07,4.081389e+05,1.492196e+03,1.014678e+05,1.775783e+05,2.014636e+04,6.864226e+04,3.581117e+04,8.953675e+03,2.835399e+04
Andorra,2.271526e+08,2.433866e+05,2.963523e+04,5.136313e+03,2.622238e+04,1.094599e+05,1.710989e+03,9.121311e+03,9.294468e+04,1.709904e+03,1.996168e+04
...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,2.996855e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Western Sahara,3.090867e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.512950e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.584995e+04
Yemen,4.868245e+09,1.299043e+08,2.435425e+06,4.701871e+05,4.506364e+06,8.210006e+06,3.325082e+06,7.883387e+05,2.869893e+06,2.196474e+05,5.475370e+06


In [ ]:
df_unprotected

,transition_0,transition_1,transition_2,transition_3,transition_4,transition_5,transition_6,transition_7,transition_8,transition_9,transition_10
country,,,,,,,,,,,
Afghanistan,6.110042e+11,1.942994e+08,1.740806e+08,7.565696e+08,2.430863e+08,5.386001e+08,6.546264e+08,1.247403e+07,1.365006e+08,2.005066e+09,2.363971e+09
Albania,2.311214e+10,1.041268e+08,4.535284e+07,2.873812e+06,1.698963e+07,7.479110e+07,8.920121e+06,1.903561e+06,8.417134e+06,4.609989e+06,2.955173e+07
Algeria,1.052304e+12,1.603326e+08,1.667298e+08,1.403458e+07,3.325844e+07,1.848154e+08,3.199420e+07,1.522731e+07,1.701609e+07,8.228351e+07,7.603286e+08
American Samoa,1.064722e+08,6.673749e+07,9.893554e+05,1.790729e+04,4.707593e+05,7.035760e+05,2.148849e+05,2.088954e+05,1.768173e+05,1.790779e+04,1.380309e+05
Andorra,3.253703e+08,3.145435e+05,3.363419e+04,6.266643e+03,3.021183e+04,9.807429e+04,1.483032e+04,3.989766e+03,1.082266e+05,1.367357e+04,1.140373e+04
...,...,...,...,...,...,...,...,...,...,...,...
Western Sahara,2.680124e+11,4.408739e+06,1.213588e+06,4.333248e+05,1.614949e+07,6.778272e+07,3.352124e+07,6.799204e+05,1.326193e+06,9.588192e+05,4.061561e+08
Western Sahara|Morocco,3.035324e+07,6.304296e+08,2.032276e+06,2.959628e+05,5.609147e+06,3.085063e+06,1.082357e+06,1.611273e+06,3.620547e+06,1.991482e+04,6.434033e+05
Yemen,4.488722e+11,5.696920e+08,5.872717e+07,9.714220e+05,8.126969e+07,2.386462e+08,6.386063e+06,1.126447e+07,2.160992e+07,4.951093e+05,3.285828e+07


In [ ]:
# close the connection
conn.close()